# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

## Code

In [359]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
import matplotlib.pyplot as plt
pd.options.plotting.backend = 'plotly'

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

### Framing the Problem

In [ ]:
# TODO

In [30]:
df = pd.read_excel('outage_copy.xlsx', header=5)
# df = df.drop([0]).drop(columns='variables').set_index('OBS')
# df_dropped = df.dropna(subset=['OUTAGE.START.TIME'])
# df_dropped = df_dropped.dropna(subset=['OUTAGE.RESTORATION.TIME'])
# temp = pd.to_timedelta(df_dropped['OUTAGE.START.TIME'].astype(str))
# df_dropped['OUTAGE.START'] = df_dropped['OUTAGE.START.DATE'] + temp
# temp = pd.to_timedelta(df_dropped['OUTAGE.RESTORATION.TIME'].astype(str))
# df_dropped['OUTAGE.RESTORATION'] = df_dropped['OUTAGE.RESTORATION.DATE'] + temp
# df_dropped['YEAR'] = df_dropped['YEAR'].astype(int)
# df_dropped['MONTH'] = df_dropped['MONTH'].astype(int)
# df_dropped = df_dropped.reset_index()
df = df.drop([0]).drop(columns='variables')
df.shape

(1534, 56)

In [163]:
df['CUSTOMERS.AFFECTED'].value_counts()
df['CUSTOMERS.AFFECTED'].value_counts()

0.0         212
50000.0      24
60000.0      12
150000.0     11
70000.0      10
           ... 
200717.0      1
64500.0       1
111555.0      1
146094.0      1
14273.0       1
Name: CUSTOMERS.AFFECTED, Length: 569, dtype: int64

In [34]:
df.columns

Index(['OBS', 'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
    

In [54]:
df['MONTH'].isna().value_counts()

False    1525
True        9
Name: MONTH, dtype: int64

### Baseline Model

In [69]:
# TODO

# relevant columns : 'OUTAGE.DURATION', 'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'TOTAL.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS', 'AREAPCT_URBAN'
model_1_df = df[['OUTAGE.DURATION', 'YEAR', 'MONTH', 'U.S._STATE', 'TOTAL.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS', 'AREAPCT_URBAN', 'CAUSE.CATEGORY']]
model_1_df = model_1_df.dropna()
model_1_df

,OUTAGE.DURATION,YEAR,MONTH,U.S._STATE,POSTAL.CODE,TOTAL.PRICE,TOTAL.SALES,TOTAL.CUSTOMERS,AREAPCT_URBAN,CAUSE.CATEGORY
1,3060,2011.0,7.0,Minnesota,MN,9.28,6562520,2595696.0,2.14,severe weather
2,1,2014.0,5.0,Minnesota,MN,9.28,5284231,2640737.0,2.14,intentional attack
3,3000,2010.0,10.0,Minnesota,MN,8.15,5222116,2586905.0,2.14,severe weather
4,2550,2012.0,6.0,Minnesota,MN,9.19,5787064,2606813.0,2.14,severe weather
5,1740,2015.0,7.0,Minnesota,MN,10.43,5970339,2673531.0,2.14,severe weather
...,...,...,...,...,...,...,...,...,...,...
1526,870,2011.0,6.0,Idaho,ID,6.91,1886871,794925.0,0.6,intentional attack
1527,0,2016.0,3.0,Idaho,ID,8.05,1657605,849763.0,0.6,intentional attack
1530,720,2011.0,12.0,North Dakota,ND,7.56,1313678,394394.0,0.27,public appeal
1532,59,2009.0,8.0,South Dakota,SD,7.67,924051,436229.0,0.3,islanding


In [157]:
sum(model_1_df['TOTAL.SALES'] == 0.0)

0

In [73]:
X_train, X_test, y_train, y_test = (
    train_test_split(model_1_df[['OUTAGE.DURATION', 'YEAR', 'MONTH', 'U.S._STATE', 'TOTAL.PRICE', 'TOTAL.SALES', 'TOTAL.CUSTOMERS', 'AREAPCT_URBAN']], model_1_df['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
)

In [74]:
round(df['TOTAL.PRICE'].max())

31

In [226]:
log_transformer = FunctionTransformer(np.log, validate=True)
neigh = KNeighborsClassifier(n_neighbors=10)
preprocessor = ColumnTransformer(
        transformers=[
            ('Log_Sales', log_transformer, ['TOTAL.SALES']),
            ('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'YEAR', 'MONTH']),
        ],
        remainder='passthrough'
    )
pipeline = Pipeline([
        ('prepocessor', preprocessor),
        ('knn', neigh)
    ])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('prepocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Log_Sales',
                                                  FunctionTransformer(func=<ufunc 'log'>,
                                                                      validate=True),
                                                  ['TOTAL.SALES']),
                                                 ('One_Hot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['U.S._STATE', 'YEAR',
                                                   'MONTH'])])),
                ('knn', KNeighborsClassifier(n_neighbors=10))])

In [227]:
pipeline.score(X_train, y_train)

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



0.6785063752276868

In [228]:
sum(pipeline.predict(X_train) == y_train)/len(X_train)

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



0.6785063752276868

In [229]:
pipeline.predict(X_train)

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



array(['severe weather', 'public appeal', 'intentional attack', ...,
       'severe weather', 'intentional attack', 'severe weather'],
      dtype=object)

In [230]:
pipeline.score(X_test, y_test)

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



0.6092896174863388

### Final Model

In [ ]:
##Standarize values

In [259]:
hyperparameters_knn = {
    'knn__n_neighbors': [1,2,3,4,5,6,7,8,9,10,15,20,25],
    'knn__weights' : ['uniform', 'distance'],
    'knn__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'knn__p': [1,2]
}
searcher = GridSearchCV(pipeline, hyperparameters_knn, cv=5)
searcher.fit(X_train, y_train)
searcher.best_params_

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.

/Users/demo/anaconda3/envs/dsc80/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True

{'knn__algorithm': 'auto',
 'knn__n_neighbors': 6,
 'knn__p': 2,
 'knn__weights': 'distance'}

In [317]:
neigh_v2 = KNeighborsClassifier(n_neighbors=6, weights='distance', algorithm='auto', p=2)
preprocessor_knn_v2 = ColumnTransformer(
        transformers=[
            #('Log_Sales', log_transformer, ['TOTAL.SALES']),
            ('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE']),
        ],
        remainder='drop'
    )
pipeline_knn_v2 = Pipeline([
        ('prepocessor', preprocessor_knn_v2),
        ('knn', neigh_v2)
    ])
pipeline_knn_v2.fit(X_train, y_train)
print('Train Error: ' + str(1-pipeline_knn_v2.score(X_train, y_train)))
print('Test Error: ' + str(1-pipeline_knn_v2.score(X_test, y_test)))


ValueError: A given column is not a column of the dataframe

In [366]:
df_1 = df.copy().drop(columns=['OBS', 'HURRICANE.NAMES', 'CAUSE.CATEGORY.DETAIL', 'DEMAND.LOSS.MW', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME', 'OUTAGE.DURATION', 'POSTAL.CODE', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME'])
df_1 = df_1.dropna()
df_X = df_1.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_1[['CAUSE.CATEGORY']]

column_name = []
tr_err = []
ts_err = []
f1_Scores_TR = []
f1_Scores_TS = []
X_train, X_test, y_train, y_test = (
        train_test_split(df_X, df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    )

for col in numeric_columns:
    #leftover = numeric_columns[:col] + numeric_columns[i+1:]
    # X_train, X_test, y_train, y_test = (
    #     train_test_split(df_X[[col]], df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    # )
    log_transformer = FunctionTransformer(np.log, validate=True)
    tree = RandomForestClassifier(criterion='entropy', max_depth=5, max_features ='log2')
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', [col])
                #('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'YEAR', 'MONTH', 'NERC.REGION', 'CLIMATE.REGION', 'CLIMATE.CATEGORY']),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ])
    pipeline_tree.fit(X_train, y_train)
    column_name.append(col + ' NUM')
    tr_err.append(1-pipeline_tree.score(X_train, y_train))
    ts_err.append(1-pipeline_tree.score(X_test, y_test))
    preds_TR = pipeline_tree.predict(X_train)
    f1_Scores_TR.append(1-f1_score(preds_TR, y_train, average = 'micro'))
    preds_TS = pipeline_tree.predict(X_test)
    f1_Scores_TS.append(1-f1_score(preds_TS, y_test, average = 'micro'))

    # print(col)
    # print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
    # print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))
print("CATEGORICAL")
for col in categorical_columns:
    # X_train, X_test, y_train, y_test = (
    #     train_test_split(df_X[[col]], df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    # )
    log_transformer = FunctionTransformer(np.log, validate=True)
    tree = RandomForestClassifier(criterion='entropy', max_depth=5, max_features ='log2')
    preprocessor_tree = ColumnTransformer(
            transformers=[
                #('column', 'passthrough', [col])
                ('One_Hot', OneHotEncoder(handle_unknown='ignore'), [col]),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ])
    pipeline_tree.fit(X_train, y_train)
    column_name.append(col + ' CAT')
    tr_err.append(1-pipeline_tree.score(X_train, y_train))
    ts_err.append(1-pipeline_tree.score(X_test, y_test))
    preds_TR = pipeline_tree.predict(X_train)
    f1_Scores_TR.append(1-f1_score(preds_TR, y_train, average = 'micro'))
    preds_TS = pipeline_tree.predict(X_test)
    f1_Scores_TS.append(1-f1_score(preds_TS, y_test, average = 'micro'))
    # print(col)
    # print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
    # print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))
errors = pd.DataFrame()
errors['column'] = column_name
errors['train_err'] = tr_err
errors['test_err'] = ts_err
errors['F1_train_err'] = f1_Scores_TR
errors['F1_test_err'] = f1_Scores_TS
errors.sort_values(by='F1_test_err')

CATEGORICAL


,column,train_err,test_err,F1_train_err,F1_test_err
3,CUSTOMERS.AFFECTED NUM,0.140881,0.214286,0.140881,0.214286
16,COM.CUSTOMERS NUM,0.260377,0.278195,0.260377,0.278195
15,RES.CUSTOMERS NUM,0.267925,0.285714,0.267925,0.285714
21,IND.CUST.PCT NUM,0.286792,0.289474,0.286792,0.289474
36,AREAPCT_URBAN NUM,0.289308,0.289474,0.289308,0.289474
35,POPDEN_RURAL NUM,0.289308,0.289474,0.289308,0.289474
31,POPPCT_URBAN NUM,0.290566,0.296992,0.290566,0.296992
37,AREAPCT_UC NUM,0.295597,0.296992,0.295597,0.296992
8,RES.SALES NUM,0.262893,0.296992,0.262893,0.296992
29,PI.UTIL.OFUSA NUM,0.305660,0.296992,0.305660,0.296992


We can see our best column is 'CUSTOMERS.AFFECTED'. Let's combine that column with other ones to see the scores we get!

In [371]:
df_1 = df.copy().drop(columns=['OBS', 'HURRICANE.NAMES', 'CAUSE.CATEGORY.DETAIL', 'DEMAND.LOSS.MW', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME', 'OUTAGE.DURATION', 'POSTAL.CODE', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME'])
df_1 = df_1.dropna()
df_X = df_1.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_1[['CAUSE.CATEGORY']]

column_name = []
tr_err = []
ts_err = []
f1_Scores_TR = []
f1_Scores_TS = []
X_train, X_test, y_train, y_test = (
        train_test_split(df_X, df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    )
col1 = ['CUSTOMERS.AFFECTED']
for col in numeric_columns:
    col1 = ['CUSTOMERS.AFFECTED']
    col1.append(col)
    #leftover = numeric_columns[:col] + numeric_columns[i+1:]
    # X_train, X_test, y_train, y_test = (
    #     train_test_split(df_X[[col]], df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    # )
    log_transformer = FunctionTransformer(np.log, validate=True)
    tree = RandomForestClassifier(max_depth=15)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1)
                #('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'YEAR', 'MONTH', 'NERC.REGION', 'CLIMATE.REGION', 'CLIMATE.CATEGORY']),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ])
    column_name.append(str(col1) + ' NUM')
    temp_train = []
    temp_test =[]
    temp_f1_tr =[]
    temp_f1_ts = []
    for i in range(10):
        pipeline_tree.fit(X_train, y_train)
        
        temp_train.append(1-pipeline_tree.score(X_train, y_train))
        temp_test.append(1-pipeline_tree.score(X_test, y_test))

        preds_TR = pipeline_tree.predict(X_train)
        temp_f1_tr.append(1-f1_score(preds_TR, y_train, average = 'weighted'))
        preds_TS = pipeline_tree.predict(X_test)
        temp_f1_ts.append(1-f1_score(preds_TS, y_test, average = 'weighted'))

    tr_err.append(np.mean(temp_train))
    ts_err.append(np.mean(temp_test))
    f1_Scores_TR.append(np.mean(temp_f1_tr))
    f1_Scores_TS.append(np.mean(temp_f1_ts))



    # print(col)
    # print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
    # print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))
print("CATEGORICAL")
for col in categorical_columns:
    col1 = ['CUSTOMERS.AFFECTED']
    # X_train, X_test, y_train, y_test = (
    #     train_test_split(df_X[[col]], df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    # )
    log_transformer = FunctionTransformer(np.log, validate=True)
    tree = RandomForestClassifier(max_depth=15)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1),
                ('One_Hot', OneHotEncoder(handle_unknown='ignore'), [col]),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ]) 
    column_name.append(str(col1) + col + ' CAT')
    temp_train = []
    temp_test =[]
    temp_f1_tr =[]
    temp_f1_ts = []
    for i in range(10):
        pipeline_tree.fit(X_train, y_train)
        
        temp_train.append(1-pipeline_tree.score(X_train, y_train))
        temp_test.append(1-pipeline_tree.score(X_test, y_test))

        preds_TR = pipeline_tree.predict(X_train)
        temp_f1_tr.append(1-f1_score(preds_TR, y_train, average = 'weighted'))
        preds_TS = pipeline_tree.predict(X_test)
        temp_f1_ts.append(1-f1_score(preds_TS, y_test, average = 'weighted'))

    tr_err.append(np.mean(temp_train))
    ts_err.append(np.mean(temp_test))
    f1_Scores_TR.append(np.mean(temp_f1_tr))
    f1_Scores_TS.append(np.mean(temp_f1_ts))
    # print(col)
    # print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
    # print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))
errors = pd.DataFrame()
errors['column'] = column_name
errors['train_err'] = tr_err
errors['test_err'] = ts_err
errors['F1_train_err'] = f1_Scores_TR
errors['F1_test_err'] = f1_Scores_TS
errors.sort_values(by='F1_test_err')

CATEGORICAL


,column,train_err,test_err,F1_train_err,F1_test_err
13,"['CUSTOMERS.AFFECTED', 'COM.PERCEN'] NUM",0.000503,0.156767,0.000500,0.135458
24,"['CUSTOMERS.AFFECTED', 'PC.REALGSP.REL'] NUM",0.007547,0.160150,0.007451,0.139462
10,"['CUSTOMERS.AFFECTED', 'IND.SALES'] NUM",0.000252,0.165789,0.000248,0.145431
4,"['CUSTOMERS.AFFECTED', 'RES.PRICE'] NUM",0.002642,0.163910,0.002614,0.146870
41,['CUSTOMERS.AFFECTED']U.S._STATE CAT,0.083396,0.169925,0.070945,0.147314
5,"['CUSTOMERS.AFFECTED', 'COM.PRICE'] NUM",0.004654,0.171429,0.004592,0.148757
20,"['CUSTOMERS.AFFECTED', 'COM.CUST.PCT'] NUM",0.009182,0.168797,0.009053,0.149309
19,"['CUSTOMERS.AFFECTED', 'RES.CUST.PCT'] NUM",0.008931,0.171429,0.008825,0.149376
11,"['CUSTOMERS.AFFECTED', 'TOTAL.SALES'] NUM",0.000000,0.165038,0.000000,0.150292
7,"['CUSTOMERS.AFFECTED', 'TOTAL.PRICE'] NUM",0.007925,0.166165,0.007692,0.155320


In [370]:
f1_score?

Signature:
f1_score(
    y_true,
    y_pred,
    *,
    labels=None,
    pos_label=1,
    average='binary',
    sample_weight=None,
    zero_division='warn',
)
Docstring:
Compute the F1 score, also known as balanced F-score or F-measure.

The F1 score can be interpreted as a harmonic mean of the precision and
recall, where an F1 score reaches its best value at 1 and worst score at 0.
The relative contribution of precision and recall to the F1 score are
equal. The formula for the F1 score is::

    F1 = 2 * (precision * recall) / (precision + recall)

In the multi-class and multi-label case, this is the average of
the F1 score of each class with weighting depending on the ``average``
parameter.

Read more in the :ref:`User Guide <precision_recall_f_measure_metrics>`.

Parameters
----------
y_true : 1d array-like, or label indicator array / sparse matrix
    Ground truth (correct) target values.

y_pred : 1d array-like, or label indicator array / sparse matrix
    Estimated targets as re

In [345]:
df_1 = df.copy().drop(columns=['OBS', 'HURRICANE.NAMES', 'CAUSE.CATEGORY.DETAIL', 'DEMAND.LOSS.MW', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME', 'OUTAGE.DURATION', 'POSTAL.CODE', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME'])
df_1 = df_1.dropna()
df_X = df_1.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_1[['CAUSE.CATEGORY']]

column_name = []
tr_err = []
ts_err = []
X_train, X_test, y_train, y_test = (
        train_test_split(df_X, df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    )
col1 = ['CUSTOMERS.AFFECTED', 'COM.PERCEN']
for col in numeric_columns:
    col1 = ['CUSTOMERS.AFFECTED', 'COM.PERCEN']
    col1.append(col)
    #leftover = numeric_columns[:col] + numeric_columns[i+1:]
    # X_train, X_test, y_train, y_test = (
    #     train_test_split(df_X[[col]], df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    # )
    log_transformer = FunctionTransformer(np.log, validate=True)
    tree = RandomForestClassifier(max_depth=15)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1)
                #('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'YEAR', 'MONTH', 'NERC.REGION', 'CLIMATE.REGION', 'CLIMATE.CATEGORY']),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ])
    column_name.append(str(col1) + ' NUM')
    temp_train = []
    temp_test =[]
    for i in range(10):
        pipeline_tree.fit(X_train, y_train)
        
        temp_train.append(1-pipeline_tree.score(X_train, y_train))
        temp_test.append(1-pipeline_tree.score(X_test, y_test))

    tr_err.append(np.mean(temp_train))
    ts_err.append(np.mean(temp_test))


    # print(col)
    # print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
    # print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))
print("CATEGORICAL")
for col in categorical_columns:
    col1 = ['CUSTOMERS.AFFECTED', 'COM.PERCEN']
    # X_train, X_test, y_train, y_test = (
    #     train_test_split(df_X[[col]], df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    # )
    log_transformer = FunctionTransformer(np.log, validate=True)
    tree = RandomForestClassifier(max_depth=15)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1),
                ('One_Hot', OneHotEncoder(handle_unknown='ignore'), [col]),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ]) 
    column_name.append(str(col1) + col + ' CAT')
    temp_train = []
    temp_test =[]
    for i in range(10):
        pipeline_tree.fit(X_train, y_train)
        
        temp_train.append(1-pipeline_tree.score(X_train, y_train))
        temp_test.append(1-pipeline_tree.score(X_test, y_test))

    tr_err.append(np.mean(temp_train))
    ts_err.append(np.mean(temp_test))
    # print(col)
    # print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
    # print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))
errors = pd.DataFrame()
errors['column'] = column_name
errors['train_err'] = tr_err
errors['test_err'] = ts_err
errors.sort_values(by='test_err')

CATEGORICAL


,column,train_err,test_err
5,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'COM.PRIC...",0.000377,0.146617
2,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000377,0.147744
23,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'PC.REALG...",0.000629,0.150000
11,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'TOTAL.SA...",0.000000,0.150376
0,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'YEAR'] NUM",0.000126,0.150752
9,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'COM.SALE...",0.000000,0.151880
4,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'RES.PRIC...",0.000252,0.152632
44,"['CUSTOMERS.AFFECTED', 'COM.PERCEN']CLIMATE.CA...",0.000126,0.153383
19,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'RES.CUST...",0.000503,0.153759
25,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'PC.REALG...",0.001132,0.154511


In [346]:
df_1 = df.copy().drop(columns=['OBS', 'HURRICANE.NAMES', 'CAUSE.CATEGORY.DETAIL', 'DEMAND.LOSS.MW', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME', 'OUTAGE.DURATION', 'POSTAL.CODE', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME'])
df_1 = df_1.dropna()
df_X = df_1.drop(columns='CAUSE.CATEGORY')
numeric_columns = df_X.columns[df_X.apply(pd.to_numeric, errors='coerce').notna().all()]
df_X[numeric_columns] = df_X[numeric_columns].apply(pd.to_numeric, errors='coerce')
numeric_columns = df_X.select_dtypes(include=['number']).columns
categorical_columns = df_X.select_dtypes(include=['object']).columns
df_Y = df_1[['CAUSE.CATEGORY']]

column_name = []
tr_err = []
ts_err = []
X_train, X_test, y_train, y_test = (
        train_test_split(df_X, df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    )
col1 = ['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY.LEVEL']
for col in numeric_columns:
    col1 = ['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY.LEVEL']
    col1.append(col)
    #leftover = numeric_columns[:col] + numeric_columns[i+1:]
    # X_train, X_test, y_train, y_test = (
    #     train_test_split(df_X[[col]], df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    # )
    log_transformer = FunctionTransformer(np.log, validate=True)
    tree = RandomForestClassifier(max_depth=15)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1)
                #('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'YEAR', 'MONTH', 'NERC.REGION', 'CLIMATE.REGION', 'CLIMATE.CATEGORY']),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ])
    column_name.append(str(col1) + ' NUM')
    temp_train = []
    temp_test =[]
    for i in range(10):
        pipeline_tree.fit(X_train, y_train)
        
        temp_train.append(1-pipeline_tree.score(X_train, y_train))
        temp_test.append(1-pipeline_tree.score(X_test, y_test))

    tr_err.append(np.mean(temp_train))
    ts_err.append(np.mean(temp_test))


    # print(col)
    # print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
    # print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))
print("CATEGORICAL")
for col in categorical_columns:
    col1 = ['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY.LEVEL']
    # X_train, X_test, y_train, y_test = (
    #     train_test_split(df_X[[col]], df_1['CAUSE.CATEGORY'],test_size=0.25, random_state=1)
    # )
    log_transformer = FunctionTransformer(np.log, validate=True)
    tree = RandomForestClassifier(max_depth=15)
    preprocessor_tree = ColumnTransformer(
            transformers=[
                ('column', 'passthrough', col1),
                ('One_Hot', OneHotEncoder(handle_unknown='ignore'), [col]),
            ],
            remainder='drop'
        )
    pipeline_tree = Pipeline([
            ('prepocessor', preprocessor_tree),
            ('rf', tree)
        ]) 
    column_name.append(str(col1) + col + ' CAT')
    temp_train = []
    temp_test =[]
    for i in range(10):
        pipeline_tree.fit(X_train, y_train)
        
        temp_train.append(1-pipeline_tree.score(X_train, y_train))
        temp_test.append(1-pipeline_tree.score(X_test, y_test))

    tr_err.append(np.mean(temp_train))
    ts_err.append(np.mean(temp_test))
    # print(col)
    # print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
    # print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))
errors = pd.DataFrame()
errors['column'] = column_name
errors['train_err'] = tr_err
errors['test_err'] = ts_err
errors.sort_values(by='test_err')

CATEGORICAL


,column,train_err,test_err
5,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000126,0.140977
12,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000252,0.142857
23,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000252,0.145489
7,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000000,0.145865
14,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000377,0.148496
38,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000126,0.148872
22,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000000,0.148872
39,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000377,0.149248
44,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000377,0.150000
20,"['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY....",0.000126,0.150376


In [357]:
errors = errors.sort_values(by='test_err')
errors['column'][:2].values

array(["['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY.LEVEL', 'COM.PRICE'] NUM",
       "['CUSTOMERS.AFFECTED', 'COM.PERCEN', 'ANOMALY.LEVEL', 'RES.PERCEN'] NUM"],
      dtype=object)

In [302]:
log_transformer = FunctionTransformer(np.log, validate=True)
tree = RandomForestClassifier(criterion='entropy', max_depth=20, max_features ='log2')
preprocessor_tree = ColumnTransformer(
        transformers=[
            ('Log_Sales', log_transformer, ['TOTAL.SALES']),
            ('One_Hot', OneHotEncoder(handle_unknown='ignore'), ['U.S._STATE', 'YEAR', 'MONTH']),
        ],
        remainder='passthrough'
    )
pipeline_tree = Pipeline([
        ('prepocessor', preprocessor_tree),
        ('rf', tree)
    ])
pipeline_tree.fit(X_train, y_train)
print('Train Error: ' + str(1-pipeline_tree.score(X_train, y_train)))
print('Test Error: ' + str(1-pipeline_tree.score(X_test, y_test)))

Train Error: 0.0018214936247723523
Test Error: 0.27595628415300544


In [294]:
hyperparameters_tree ={
    #'rf__n_estimators': [10, 50, 100, 200, 300, 400, 500, 1000],
    'rf__criterion': ["gini", "entropy"],
    'rf__max_depth': [10, 15, 20, 25, 50],
    'rf__min_samples_split': [1, 2, 3, 4, 5, 8, 10],
    #'rf__min_samples_leaf': [1, 2, 4, 6, 8, 10],
    'rf__max_features': ["auto", "log2", None]
}
searcher_tree = GridSearchCV(pipeline_tree, hyperparameters_tree, cv=5)
searcher_tree.fit(X_train, y_train)
searcher_tree.best_params_

{'rf__criterion': 'entropy',
 'rf__max_depth': 20,
 'rf__max_features': 'log2',
 'rf__min_samples_split': 4}

In [304]:
searcher_tree.cv_results_['mean_test_score']

array([0.70582814, 0.70766293, 0.70491075, 0.70217933, 0.7049066 ,
       0.70219593, 0.7003653 , 0.69854711, 0.69672063, 0.69671648,
       0.69124118, 0.69034039, 0.70400581, 0.70400166, 0.70582814,
       0.70400581, 0.70580739, 0.70490245, 0.72403487, 0.71948111,
       0.71766708, 0.71129099, 0.71127439, 0.71402657, 0.71765878,
       0.71127439, 0.7158406 , 0.70582399, 0.70581569, 0.70582399,
       0.71401411, 0.70855127, 0.70672478, 0.7049066 , 0.70580739,
       0.7049066 , 0.72494396, 0.71858032, 0.72039435, 0.71036945,
       0.71129099, 0.70945205, 0.72129929, 0.72040266, 0.72039435,
       0.71220008, 0.70946451, 0.71036945, 0.70582399, 0.70401411,
       0.70401411, 0.70035284, 0.70035284, 0.70581154, 0.71220423,
       0.72403902, 0.72040266, 0.71492736, 0.71310502, 0.70854711,
       0.71674969, 0.7213159 , 0.71856787, 0.71402242, 0.71220423,
       0.70034869, 0.7058323 , 0.70855542, 0.70948111, 0.70580739,
       0.7049149 , 0.70489415, 0.71220839, 0.70672893, 0.70127

In [ ]:
# TODO

### Fairness Analysis

In [ ]:
# TODO